In [1]:
# version 0415:   add feature [ip_test_hh_app, ip_test_hh_os, ip_test_hh_dev, ip_test_hh_app_os]
#                 categorical use [in_test_hh] 
#                 use all date

In [2]:
VALIDATE = True

MAX_ROUNDS = 1000
EARLY_STOP = 100
OPT_ROUNDS = 680

path = '../input/'
FULL_OUTFILE = '../output/lgbt0413_dev_new.csv'
VALID_OUTFILE = '../output/lgbt0413_dev_new_vaild.csv'

import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split 
import lightgbm as lgb
from sklearn.utils import shuffle  

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

print('load train...')
train_cols = ['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
train_df = pd.read_csv(path + "train.csv", skiprows=range(1, 84903891), nrows=100000000, dtype=dtypes,
                       usecols=train_cols)
# train_df = pd.read_csv(path+"train.csv", dtype=dtypes, usecols=train_cols)
train_df = shuffle(train_df)

load train...


In [3]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed
49970821,50852,2,1,17,236,2017-11-09 00:53:51,0
44491540,27627,18,1,14,134,2017-11-08 22:55:48,0
8558177,5281,3,1,15,280,2017-11-08 07:29:01,0
51507371,73156,8,1,13,145,2017-11-09 01:23:03,0
96275713,33808,2,1,19,469,2017-11-09 14:47:13,0


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000000 entries, 49970821 to 9491816
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       object
is_attributed    uint8
dtypes: object(1), uint16(4), uint32(1), uint8(1)
memory usage: 2.7+ GB


In [5]:
import gc
gc.collect()

len_train = len(train_df)
most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
least_freq_hours_in_test_data = [6, 11, 15]

In [6]:
def prep_data(train_df):
    print('data prep...')
    most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
    least_freq_hours_in_test_data = [6, 11, 15]
    
    train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
    train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
    train_df['weekday'] = pd.to_datetime(train_df.click_time).dt.weekday.astype('uint8')

    train_df.drop(['click_time'], axis=1, inplace=True)
    gc.collect()

    train_df['in_test_hh'] = (3 
                             - 2 * train_df['hour'].isin(most_freq_hours_in_test_data) 
                             - 1 * train_df['hour'].isin(least_freq_hours_in_test_data)).astype('uint8')

    print('group by : ip, weekday, test_hh')
    gp = train_df[['ip', 'weekday', 'in_test_hh', 'channel']]\
        .groupby(by=['ip', 'weekday', 'in_test_hh'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_wday_test_hh'})
    train_df = train_df.merge(gp, on=['ip','weekday', 'in_test_hh'], how='left')
    del gp

    print('group by : ip, weekday, hour')
    gp = train_df[['ip', 'weekday', 'hour', 'channel']]\
        .groupby(by=['ip', 'weekday', 'hour'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_wday_hour'})
    train_df = train_df.merge(gp, on=['ip','weekday', 'hour'], how='left')
    del gp

    print('group by : ip, weekday, hour, os')
    gp = train_df[['ip', 'weekday', 'hour', 'os', 'channel']]\
        .groupby(by=['ip', 'weekday', 'hour', 'os'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_wday_hour_os'})
    train_df = train_df.merge(gp, on=['ip','weekday', 'hour', 'os'], how='left')
    del gp

    print('group by : ip, weekday, hour, app')
    gp = train_df[['ip', 'weekday', 'hour', 'app', 'channel']]\
        .groupby(by=['ip', 'weekday', 'hour', 'app'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_wday_hour_app'})
    train_df = train_df.merge(gp, on=['ip','weekday', 'hour', 'app'], how='left')
    del gp

    print('group by : ip, weekday, hour, app, os')
    gp = train_df[['ip', 'weekday', 'hour', 'app', 'os', 'channel']]\
        .groupby(by=['ip', 'weekday', 'hour', 'app', 'os'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_wday_hour_app_os'})
    train_df = train_df.merge(gp, on=['ip','weekday', 'hour', 'app', 'os'], how='left')
    del gp
    
    print('group by : ip, in_test_hh, app')
    gp = train_df[['ip', 'in_test_hh', 'app', 'channel']]\
        .groupby(by=['ip', 'in_test_hh', 'app'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_test_hh_app'})
    train_df = train_df.merge(gp, on=['ip', 'in_test_hh', 'app'], how='left')
    del gp
    
    print('group by : ip, in_test_hh, os')
    gp = train_df[['ip', 'in_test_hh', 'os', 'channel']]\
        .groupby(by=['ip', 'in_test_hh','os'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_test_hh_os'})
    train_df = train_df.merge(gp, on=['ip', 'in_test_hh', 'os'], how='left')
    del gp
    
    print('group by : ip, in_test_hh, device')
    gp = train_df[['ip', 'in_test_hh', 'device', 'channel']]\
        .groupby(by=['ip', 'in_test_hh', 'device'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_test_hh_dev'})
    train_df = train_df.merge(gp, on=['ip', 'in_test_hh', 'device'], how='left')
    del gp
    
    print('group by : ip, in_test_hh, app, os')
    gp = train_df[['ip', 'in_test_hh', 'app', 'os', 'channel']]\
        .groupby(by=['ip', 'in_test_hh', 'app', 'os'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'ip_test_hh_app_os'})
    train_df = train_df.merge(gp, on=['ip', 'in_test_hh', 'app', 'os'], how='left')
    del gp
    
    print('group by : app, hour, os')
    gp = train_df[['app', 'hour', 'os', 'channel']]\
        .groupby(by=['app', 'hour', 'os'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'app_hour_os'})
    train_df = train_df.merge(gp, on=['app', 'hour', 'os'], how='left')
    del gp
    
    print('group by : app, hour, device')
    gp = train_df[['app', 'hour', 'device', 'channel']]\
        .groupby(by=['app', 'hour', 'device'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'app_hour_device'})
    train_df = train_df.merge(gp, on=['app', 'hour', 'device'], how='left')
    del gp
    
    print('group by : app, hour, os, device')
    gp = train_df[['app', 'hour', 'os', 'device', 'channel']]\
        .groupby(by=['app', 'hour', 'os', 'device'])[['channel']]\
        .count().reset_index()\
        .rename(index=str, columns={'channel': 'app_hour_os_dev'})
    train_df = train_df.merge(gp, on=['app', 'hour', 'os', 'device'], how='left')
    del gp
    
    gc.collect()
#     train_df.drop(['day', 'weekday', 'in_test_hh'], axis=1, inplace=True)
    return train_df

In [7]:
# def group_feature(group, features):
#     # prep
#     print('### merge new feature...')
#     anno = 'group by: '
#     name = ''
#     for n in group.columns:
#         anno = anno + n + ', ' 
#         name = n + '_'
#     name = name[:-1]
#     print(anno)
#     print(name)
        
#     # merge
#     gp = features\
#         .groupby(group.columns)[['channel']]\
#         .count().reset_index()\
#         .rename(index=str, columns={'channel': name})
#     train_df = train_df.merge(gp, on=[group.columns], how='left')
#     del gp
#     gc.collect()

In [8]:
# def prep_data(train_df):
#     print('data prep...')
#     most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
#     least_freq_hours_in_test_data = [6, 11, 15]
    
#     train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
#     train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
#     train_df['weekday'] = pd.to_datetime(train_df.click_time).dt.weekday.astype('uint8')

#     train_df.drop(['click_time'], axis=1, inplace=True)
#     gc.collect()

#     train_df['in_test_hh'] = (3 
#                              - 2 * train_df['hour'].isin(most_freq_hours_in_test_data) 
#                              - 1 * train_df['hour'].isin(least_freq_hours_in_test_data)).astype('uint8')
#     group_feature(train_df[['ip', 'weekday', 'in_test_hh']], 
#                   train_df[['ip', 'weekday', 'in_test_hh', 'channel']])
#     group_feature(train_df[['ip', 'weekday', 'hour']], 
#                   train_df[['ip', 'weekday', 'hour', 'channel']])
#     group_feature(train_df[['ip', 'weekday', 'hour', 'os']], 
#                   train_df[['ip', 'weekday', 'hour', 'os', 'channel']])
#     group_feature(train_df[['ip', 'weekday', 'hour', 'app']], 
#                   train_df[['ip', 'weekday', 'hour', 'app', 'channel']])
#     group_feature(train_df[['ip', 'weekday', 'hour', 'app', 'os']], 
#                   train_df[['ip', 'weekday', 'hour', 'app', 'os', 'channel']])
#     group_feature(train_df[['ip', 'in_test_hh', 'app']], 
#                   train_df[['ip', 'in_test_hh', 'app', 'channel']])
#     group_feature(train_df[['ip', 'in_test_hh','os']], 
#                   train_df[['ip', 'in_test_hh','os', 'channel']])
#     group_feature(train_df[['ip', 'in_test_hh', 'device']], 
#                   train_df[['ip', 'in_test_hh', 'device', 'channel']])
#     group_feature(train_df[['ip', 'in_test_hh', 'app', 'os']], 
#                   train_df[['ip', 'in_test_hh', 'app', 'os', 'channel']])
#     return train_df

In [9]:
train_df = prep_data(train_df)
train_df.head()

data prep...
group by : ip, weekday, test_hh
group by : ip, weekday, hour
group by : ip, weekday, hour, os
group by : ip, weekday, hour, app
group by : ip, weekday, hour, app, os
group by : ip, in_test_hh, app
group by : ip, in_test_hh, os
group by : ip, in_test_hh, device
group by : ip, in_test_hh, app, os
group by : app, hour, os
group by : app, hour, device
group by : app, hour, os, device


,ip,app,device,os,channel,is_attributed,hour,day,weekday,in_test_hh,...,ip_wday_hour_os,ip_wday_hour_app,ip_wday_hour_app_os,ip_test_hh_app,ip_test_hh_os,ip_test_hh_dev,ip_test_hh_app_os,app_hour_os,app_hour_device,app_hour_os_dev
0,50852,2,1,17,236,0,0,9,3,3,...,39,16,1,37,62,352,2,16278,322628,15628
1,27627,18,1,14,134,0,22,8,2,3,...,17,12,1,351,121,3836,7,1189,100902,1185
2,5281,3,1,15,280,0,7,8,2,3,...,15,103,4,744,99,2997,25,33287,1307449,32649
3,73156,8,1,13,145,0,1,9,3,3,...,10,1,1,28,100,662,6,6953,26819,6220
4,33808,2,1,19,469,0,14,9,3,1,...,58,10,4,312,766,2825,71,183315,713719,175754


In [10]:
metrics = 'auc'
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':metrics,
        'learning_rate': 0.1,
        'num_leaves': 7,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 3,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.8,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'nthread': 8,
        'verbose': 0,
        'scale_pos_weight':99.7, # because training data is extremely unbalanced 
        'metric':metrics
}

target = 'is_attributed'
predictors = ['app','device','os', 'channel', 'hour', 'ip', 'day', 
              'weekday', 'in_test_hh',
              'ip_wday_test_hh', 'ip_wday_hour', 'ip_wday_hour_os', 'ip_wday_hour_app', 'ip_wday_hour_app_os',
              'ip_test_hh_app', 'ip_test_hh_os', 'ip_test_hh_dev', 'ip_test_hh_app_os']
categorical = ['app','device','os', 'channel', 'in_test_hh', 'ip', 'day']

print(train_df.head(5))

if VALIDATE:
    train_df, val_df = train_test_split(train_df, train_size=.95, shuffle=False )
    print(train_df.info())
    print(val_df.info())
    print("train size: ", len(train_df))
    print("valid size: ", len(val_df))
    gc.collect()
    print("Training...")
    num_boost_round=MAX_ROUNDS
    early_stopping_rounds=EARLY_STOP
    xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical)
    del train_df
    gc.collect()
    xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical)
    del val_df
    gc.collect()
    evals_results = {}
    bst = lgb.train(lgb_params, 
                     xgtrain, 
                     valid_sets= [xgvalid], 
                     valid_names=['valid'], 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10, 
                     feval=None)
    n_estimators = bst.best_iteration
    print("\nModel Report")
    print("n_estimators : ", n_estimators)
    print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
    outfile = VALID_OUTFILE
    del xgvalid
else:
    print(train_df.info())
    print("train size: ", len(train_df))
    gc.collect()
    print("Training...")
    num_boost_round=OPT_ROUNDS
    xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical
                          )
    del train_df
    gc.collect()
    bst = lgb.train(lgb_params, 
                     xgtrain, 
                     num_boost_round=num_boost_round,
                     verbose_eval=10, 
                     feval=None)
    outfile = FULL_OUTFILE
del xgtrain
gc.collect()

print('load test...')
test_cols = ['ip','app','device','os', 'channel', 'click_time', 'click_id']
test_df = pd.read_csv(path+"test.csv", dtype=dtypes, usecols=test_cols)
test_df = prep_data(test_df)
gc.collect()

sub = pd.DataFrame()
sub['click_id'] = test_df['click_id']

print("Predicting...")
sub['is_attributed'] = bst.predict(test_df[predictors])
print("writing...")
sub.to_csv(outfile, index=False, float_format='%.9f')
print("done...")
print(sub.info())

      ip  app  device  os  channel  is_attributed  hour  day  weekday  \
0  50852    2       1  17      236              0     0    9        3   
1  27627   18       1  14      134              0    22    8        2   
2   5281    3       1  15      280              0     7    8        2   
3  73156    8       1  13      145              0     1    9        3   
4  33808    2       1  19      469              0    14    9        3   

   in_test_hh       ...         ip_wday_hour_os  ip_wday_hour_app  \
0           3       ...                      39                16   
1           3       ...                      17                12   
2           3       ...                      15               103   
3           3       ...                      10                 1   
4           1       ...                      58                10   

   ip_wday_hour_app_os  ip_test_hh_app  ip_test_hh_os  ip_test_hh_dev  \
0                    1              37             62             352   


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 95000000 entries, 0 to 94999999
Data columns (total 22 columns):
ip                     uint32
app                    uint16
device                 uint16
os                     uint16
channel                uint16
is_attributed          uint8
hour                   uint8
day                    uint8
weekday                uint8
in_test_hh             uint8
ip_wday_test_hh        int64
ip_wday_hour           int64
ip_wday_hour_os        int64
ip_wday_hour_app       int64
ip_wday_hour_app_os    int64
ip_test_hh_app         int64
ip_test_hh_os          int64
ip_test_hh_dev         int64
ip_test_hh_app_os      int64
app_hour_os            int64
app_hour_device        int64
app_hour_os_dev        int64
dtypes: int64(12), uint16(4), uint32(1), uint8(5)
memory usage: 10.7 GB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 95000000 to 99999999
Data columns (total 22 columns):
ip                     uint32
app           

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid's auc: 0.954857
[20]	valid's auc: 0.961429
[30]	valid's auc: 0.966453
[40]	valid's auc: 0.968562
[50]	valid's auc: 0.970553
[60]	valid's auc: 0.971529
[70]	valid's auc: 0.972477
[80]	valid's auc: 0.972931
[90]	valid's auc: 0.973433
[100]	valid's auc: 0.973741
[110]	valid's auc: 0.973961
[120]	valid's auc: 0.974245
[130]	valid's auc: 0.974317
[140]	valid's auc: 0.974412
[150]	valid's auc: 0.974528
[160]	valid's auc: 0.974663
[170]	valid's auc: 0.974811
[180]	valid's auc: 0.97488
[190]	valid's auc: 0.975006
[200]	valid's auc: 0.975081
[210]	valid's auc: 0.975187
[220]	valid's auc: 0.975214
[230]	valid's auc: 0.975257
[240]	valid's auc: 0.975323
[250]	valid's auc: 0.975432
[260]	valid's auc: 0.975482
[270]	valid's auc: 0.975527
[280]	valid's auc: 0.975546
[290]	valid's auc: 0.975596
[300]	valid's auc: 0.975653
[310]	valid's auc: 0.975709
[320]	valid's auc: 0.975752
[330]	valid's auc: 0.975892
[340]	valid's auc: 0.97